# Introduction and Creation of Required Databases

This project relies on the following four databases:

1. **`metadata.db`**: This database contains essential information about image patches, including their working status, metadata retrieved from Google Earth Engine, and other relevant details.

2. **`annotation.db`**: This database stores the captions generated during the project. It also includes metadata about the LLM annotator, prompts used, OSM IDs, and their mappings.

3. **`osm.db`**: This database serves as a cache for OpenStreetMap (OSM) data, facilitating asynchronous captioning and efficient OSM querying processes.

4. **`annotation_meta.db`**: This database is specific to Task 3. It holds example captions and their corresponding revisions.

In [1]:
DB_ROOT = '../database'

## Create database for image patch metadata
We store all image patches and their corresponding metadata into an sqlite3 database.

Three tables are related to the downloading task, they are `collection`, `image`, and `patch` respectively.

The definitions of the three tables are given below, we use `*` to mark the primary keys: 

### collection

| ID* | NAME |
|---|---|
| INTEGER |TEXT|

**ID**: PRIMARY KEY AUTOINCREMENT

This is the identifier for the collection

**NAME**: NOT NULL UNIQUE

This is the name of the collection, which should be the name for `ee.ImageCollection()`

### image

| ID* | NAME | YEAR | VERSION | TIMESTAMP | FOOTPRINT | ASSET_SIZE | BANDS | SCALE | DIMENSION_H | DIMENSION_W | CRS | COLLECTION | SAMPLE_INTERVAL |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| INTEGER |TEXT|INTEGER|INTEGER|TEXT|TEXT|INTEGER|TEXT|REAL|INTEGER|INTEGER|TEXT|TEXT| FLOAT |

**ID**: PRIMARY KEY AUTOINCREMENT

This is the identifier for the image

**NAME**: NOT NULL UNIQUE

This is the name of the image. For images in `USDA/NAIP/DOQQ`, this information lies in the end of `id` or `system:index`.

**YEAR**:

This stores the `year` key of the image metadata. Not necessary when `TIMESTAMP` is available.

**VERSION**:

This stores the `version` key of the image metadata.

**TIMESTAMP**: NOT NULL

This is  the timestamp when the image was created. Data is taken from the `system:time_start` attribute of the image metadata. We store timestamp as an string with a format of `YYYY-MM-DD HH-MM-SS`

**FOOTPRINT**: NOT NULL

This is the boundary geometry of the image. Data is taken from the `system:footprint` attribute of the image metadata. We store the boundary geometry as a GeoJSON formatted string. The CRS of it is `EPSG:4326` by default.

**ASSET_SIZE**:

The file size of the image in Bytes.

**BANDS**:

Avaiable bands of the image. Not all bands are sampled into patches.

**SCALE**: NOT NULL

The resolution of the image in meters.(The length of a pixel cell)

**DIMENSION_H**:

The height of the image in pixels.

**DIMENSION_W**:

The width of the image in pixels.

**CRS**: NOT NULL

The CRS of the image.

**COLLECTION**: NOT NULL

This is the collection name of the image. This is used to relate the `collection` table with the `image` table.

**SAMPLE_INTERVAL**:

The interval between two adjacent patches in the image. This is used to generate the mersh grid.

### patch

| ID* | NAME | IND_X | IND_Y | DIMENSION | SAMPLE_CENTER_X | SAMPLE_CENTER_Y | BANDS | CRS | FILE_FORMAT | ASSET_SIZE | DOWNLOAD_TIME | IMAGE_NAME |
|---|---|---|---|---|---|---|---|---|---|---|---|---|
| INTEGER |TEXT|INTEGER|INTEGER|INTEGER|REAL|REAL|TEXT|TEXT|TEXT|INTEGER|TEXT|TEXT|

**ID**: PRIMARY KEY AUTOINCREMENT

This is the identifier for the patch

**NAME**: NOT NULL

The name of the patch. This can be replicative. By default, the name of the patch is `{IND_X}_{IND_Y}`.

**IND_X**: 

The patch is sampled from the original image with a mersh grid sampling. The code for generating the mersh grid and the index is given by the following code snippet. We can infer the coordinates of the patcd edges from the indexes.

```python
x, y = np.round(np.meshgrid(np.arange(lonmin, lonmax, offset_x),
                            np.arange(latmin, latmax, offset_y)),
                            7)
x_ind, y_ind = np.meshgrid(np.arange(x.shape[1]), np.arange(x.shape[0]))
```

**IND_Y**: 

Same as above.

**DIMENSION**: NOT NULL

As we cropped all the patches with a square shape, the width and height of the patch are the same. The image size is defined as $DIMENSION \times DIMENSION$

**SAMPLE_CENTER_X**: NOT NULL

This is the x position of the patch center. Its CRS is defined in `CRS`.

**SAMPLE_CENTER_Y**: NOT NULL

This is the y position of the patch center. Its CRS is defined in `CRS`.

**BANDS**: NOT NULL

The bands used in this patch. Valid values are `RGB`, `Infrared`, `SAR` and etc.

**CRS**: NOT NULL

The CRS used to sample this patch. Usually, this is `ESPG:4326`.

**FILE_FORMAT**: NOT NULL

The way this patch is written in the disk. Valid values are `JPEG`, `PNG`, `GeoTiff`, and `BMP`.

**ASSET_SIZE**:

The disk space this patch occupies in Bytes.

**DOWNLOAD_TIME**: NOT NULL

The time this patch is written in the disk. Format is `YYYY-MM-DD HH-MM-SS`.

**IMAGE_NAME**: NOT NULL

The name of the image where is patch is cropped. This column is used to link the `image` table.

In [2]:
import os
import sqlite3

# check if the folder exists, otherwise create it
if not os.path.exists(DB_ROOT):
    os.mkdir(DB_ROOT)

# init the data base
database_file = os.path.join(DB_ROOT,'metadata.db')
conn = sqlite3.connect(database_file)

# create the three tables if the table is not already created

COLLECTION_TABLE_INIT_CMD ="""
                            CREATE TABLE IF NOT EXISTS collection(
                            ID      INTEGER     PRIMARY KEY AUTOINCREMENT,
                            NAME    TEXT        NOT NULL UNIQUE 
                        )"""
IMAGE_TABLE_INIT_CMD ="""
                        CREATE TABLE IF NOT EXISTS image(
                        ID          INTEGER     PRIMARY KEY AUTOINCREMENT,
                        NAME        TEXT        NOT NULL UNIQUE,
                        YEAR        INTEGER,
                        VERSION     INTEGER,
                        TIMESTAMP   INTEGER     NOT NULL,
                        FOOTPRINT   TEXT        NOT NULL,
                        ASSET_SIZE  INTEGER,
                        BANDS       TEXT,
                        SCALE       REAL        NOT NULL,
                        DIMENSION_H INTEGER,
                        DIMENSION_W INTEGER,
                        CRS TEXT    NOT NULL,
                        COLLECTION  TEXT        NOT NULL,
                        SAMPLE_INTERVAL FLOAT,
                        FOREIGN KEY (COLLECTION) REFERENCES collection(NAME)
                    )"""
PATCH_TABLE_INIT_CMD ="""
                        CREATE TABLE IF NOT EXISTS patch(
                        ID              INTEGER     PRIMARY KEY AUTOINCREMENT,
                        NAME            TEXT        NOT NULL,
                        IND_X           INTEGER,
                        IND_Y           INTEGER,
                        DIMENSION       INTEGER     NOT NULL,
                        SAMPLE_CENTER_X REAL        NOT NULL,
                        SAMPLE_CENTER_Y REAL        NOT NULL,
                        BANDS           TEXT        NOT NULL,
                        CRS             TEXT        NOT NULL,
                        FILE_FORMAT     TEXT        NOT NULL,
                        ASSET_SIZE      INTEGER,
                        DOWNLOAD_TIME   TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                        IMAGE_NAME      TEXT        NOT NULL,
                        NUM_MAP_ELEMENTS INTEGER,
                        NUM_ANNOTATIONS INTEGER,
                        STATUS          INTEGER,
                        FOREIGN KEY (IMAGE_NAME) REFERENCES image(NAME)
                    )"""
c = conn.cursor()
c.execute(COLLECTION_TABLE_INIT_CMD)
print("collection table initialized")
c.execute(IMAGE_TABLE_INIT_CMD)
print("image table initialized")
c.execute(PATCH_TABLE_INIT_CMD)
print("patch table initialized")

# support NAIP for now
DATASET='USDA/NAIP/DOQQ'

# insert current collection information
try:
    c.execute("INSERT OR IGNORE INTO collection (NAME) VALUES ('{}')".format(DATASET))
except Exception as exc:
    print('Inserting COLLECTION {} encounters an error! {}'.format(DATASET, str(exc)))
else:
    print("COLLECTION {} inserted successfully".format(DATASET))

conn.commit()
conn.close()

collection table initialized
image table initialized
patch table initialized
COLLECTION USDA/NAIP/DOQQ inserted successfully


# Create annotation database for annotation data management

## Create database for annotation metadata
We store annotation-related metadata in an SQLite3 database. This database includes details about prompts, annotators, annotations, and their mappings.

Five tables are used in this schema: `prompt`, `annotator`, `annotation`, `annotator_prompt`, and `annotation_osm`.

The definitions of these tables are as follows:

### prompt

| ID* | DESCRIPTION | PROMPT | TYPE | CREATED_AT |
|---|---|---|---|---|
| INTEGER | TEXT | TEXT | INTEGER | TIMESTAMP |

**ID**: PRIMARY KEY AUTOINCREMENT  
This is the identifier for the prompt.

**DESCRIPTION**:  
An optional description of the prompt.

**PROMPT**: NOT NULL  
The text of the prompt used in the annotation process.

**TYPE**: NOT NULL  
The type of the prompt. Valid values are:  
- `2`: Distinctive area  
- `3`: Distinctive non-area  
- `11`: Caption revision

**CREATED_AT**:  
The timestamp when the prompt was created. The default format is `YYYY-MM-DD HH:MM:SS`.

---

### annotator

| ID* | ANNOTATOR | SETTING | CREATED_AT |
|---|---|---|---|
| INTEGER | TEXT | TEXT | TIMESTAMP |

**ID**: PRIMARY KEY AUTOINCREMENT  
This is the identifier for the annotator.

**ANNOTATOR**: NOT NULL  
The name of the annotator, typically in a Hugging Face-compatible format.

**SETTING**:  
Details about the implementation or configuration of the annotator.

**CREATED_AT**:  
The timestamp when the annotator record was created. The default format is `YYYY-MM-DD HH:MM:SS`.

---

### annotation

| ID* | PATCH | ANNOTATION | NUM_ELEMS | ANNOTATOR | PROMPT | CREATED_AT |
|---|---|---|---|---|---|---|
| INTEGER | INTEGER | TEXT | INTEGER | INTEGER | INTEGER | TIMESTAMP |

**ID**: PRIMARY KEY AUTOINCREMENT  
This is the identifier for the annotation.

**PATCH**: NOT NULL  
The ID of the patch being annotated.

**ANNOTATION**: NOT NULL  
The annotation result.

**NUM_ELEMS**: NOT NULL  
The number of elements in the annotation.

**ANNOTATOR**: NOT NULL  
The ID of the annotator who created this annotation. This references the `annotator` table.

**PROMPT**: NOT NULL  
The ID of the prompt used for this annotation. This references the `prompt` table.

**CREATED_AT**:  
The timestamp when the annotation was created. The default format is `YYYY-MM-DD HH:MM:SS`.

---

### annotator_prompt

| ID* | ANNOTATOR | PROMPT | CREATED_AT |
|---|---|---|---|
| INTEGER | INTEGER | INTEGER | TIMESTAMP |

**ID**: PRIMARY KEY AUTOINCREMENT  
This is the identifier for the annotator-prompt mapping.

**ANNOTATOR**: NOT NULL  
The ID of the annotator. This references the `annotator` table.

**PROMPT**: NOT NULL  
The ID of the prompt. This references the `prompt` table.

**CREATED_AT**:  
The timestamp when the mapping was created. The default format is `YYYY-MM-DD HH:MM:SS`.

---

### annotation_osm

| ID* | ANNOTATION | OSM_ID |
|---|---|---|
| INTEGER | INTEGER | INTEGER |

**ID**: PRIMARY KEY AUTOINCREMENT  
This is the identifier for the annotation-OSM mapping.

**ANNOTATION**: NOT NULL  
The ID of the annotation. This references the `annotation` table.

**OSM_ID**: NOT NULL  
The OpenStreetMap (OSM) ID related to the annotation.

---


In [3]:
# table for prompts
# TYPE: 2: Distinctive area, 3: Distinctive none-area
ANNOTATION_PROMPT_TABLE_INIT_CMD = """create table if not exists prompt (
                                    ID          INTEGER     PRIMARY KEY AUTOINCREMENT,
                                    DESCRIPTION TEXT,
                                    PROMPT      TEXT        NOT NULL,
                                    TYPE        INTEGER     NOT NULL,
                                    CREATED_AT  TIMESTAMP   DEFAULT (datetime('now','localtime'))
                                    )"""

# table for annotators
# ANNOTATOR: annotator name in huggingface format, SETTING: detailed implementation of the annotator
ANNOTATION_ANNOTATOR_TABLE_INIT_CMD = """create table if not exists annotator (
                                    ID          INTEGER     PRIMARY KEY AUTOINCREMENT,
                                    ANNOTATOR   TEXT        NOT NULL,
                                    SETTING     TEXT,
                                    CREATED_AT  TIMESTAMP   DEFAULT (datetime('now','localtime'))
                                    )"""

# table for annotations
# ANNOTATION: annotation result
ANNOTATION_ANNOTATION_TABLE_INIT_CMD = """create table if not exists annotation (
                                    ID          INTEGER     PRIMARY KEY AUTOINCREMENT,
                                    PATCH       INTEGER     NOT NULL,
                                    ANNOTATION  TEXT        NOT NULL,
                                    NUM_ELEMS   INTEGER     NOT NULL,
                                    ANNOTATOR   INTEGER     NOT NULL,
                                    PROMPT      INTEGER     NOT NULL,
                                    CREATED_AT  TIMESTAMP   DEFAULT (datetime('now','localtime')),
                                    FOREIGN KEY (ANNOTATOR) REFERENCES annotator(ID),
                                    FOREIGN KEY (PROMPT)    REFERENCES prompt(ID)
                                    )"""
                                    
# table for annotator-prompt mapping
# ANNOTATOR_PROMPT: mapping between annotator and prompt
ANNOTATION_ANNOTATOR_PROMPT_TABLE_INIT_CMD = """create table if not exists annotator_prompt (
                                    ID          INTEGER     PRIMARY KEY AUTOINCREMENT,
                                    ANNOTATOR   INTEGER     NOT NULL,
                                    PROMPT      INTEGER     NOT NULL,
                                    CREATED_AT  TIMESTAMP   DEFAULT (datetime('now','localtime')),
                                    FOREIGN KEY (ANNOTATOR) REFERENCES annotator(ID),
                                    FOREIGN KEY (PROMPT)    REFERENCES prompt(ID)
                                    )"""

# table for annotation-osm mapping
# ANNOTATION_OSM: mapping between annotation and osm id
ANNOTATION_ANNOTATION_OSM_TABLE_INIT_CMD = """create table if not exists annotation_osm (
                                    ID          INTEGER     PRIMARY KEY AUTOINCREMENT,
                                    ANNOTATION  INTEGER     NOT NULL,
                                    OSM_ID      INTEGER     NOT NULL,
                                    FOREIGN KEY (ANNOTATION) REFERENCES annotation(ID)
                                    )"""
                                    
# init the data base
database_file = os.path.join(DB_ROOT,'annotation.db')
conn = sqlite3.connect(database_file)

c_annotation = conn.cursor()
c_annotation.execute(ANNOTATION_ANNOTATION_TABLE_INIT_CMD)
c_annotation.execute(ANNOTATION_ANNOTATOR_TABLE_INIT_CMD)
c_annotation.execute(ANNOTATION_PROMPT_TABLE_INIT_CMD)
c_annotation.execute(ANNOTATION_ANNOTATOR_PROMPT_TABLE_INIT_CMD)
c_annotation.execute(ANNOTATION_ANNOTATION_OSM_TABLE_INIT_CMD)

conn.commit()
conn.close()

# Create OSM database for cache OSM data

> **Note:** This notebook is not designed to be run directly. Its purpose is to generate the database schema and documentation for the OSM database used in the RSTeller project. The database will be created automatically when `download/osm_data_downloader.py` is run for the first time.

## Create database for OpenStreetMap (OSM) data
This database is used to store metadata related to OSM elements and their association with image patches. The schema includes two tables: `osm_data` and `osm_background`.

### Notes on OSM Query Methodology
The OSM data is retrieved using a two-step method:  
1. **Bounding Box Query**: We first query ways and relations using a bounding box filter to capture all geometries that satisfy the filter's policy. This step ensures the retrieval of essential geo-elements but may miss larger enclosing geometries.
2. **`is_in` Query**: To address the gaps, an additional query is performed using the `is_in` operator, targeting geometries tagged as `area`. This identifies elements enclosing the center point of the image patch, providing a complete representation of the image content.

### Storage Policy
- The `osm_data` table stores OSM IDs for elements queried via `is_in`. Their tags are stored separately in the `osm_background` table.
- Geometry data for elements queried via `is_in` is not stored to save storage space and improve performance, as these geometries are often large and shared across multiple patches.

---

### Table Definitions

#### osm_data
| ID* | PATCH_NAME | OSM_ID | TAGS | TYPE | GEOMETRY | DATA_DATE |
|---|---|---|---|---|---|---|
| INTEGER | TEXT | INTEGER | TEXT | TINYINT | TEXT | DATE |

**ID**: PRIMARY KEY AUTOINCREMENT  
The unique identifier for the OSM data record.

**PATCH_NAME**: NOT NULL  
The name of the patch associated with this OSM element.

**OSM_ID**: NOT NULL  
The unique identifier of the OSM element.

**TAGS**:  
Tags associated with the OSM element in JSON format.

**TYPE**: NOT NULL  
The type of OSM element. Valid values are:  
- `0`: Background  
- `1`: Area  
- `2`: Non-area  

**GEOMETRY**:  
The geometry of the OSM element in GeoJSON format.

**DATA_DATE**:  
The date associated with the OSM data.

---

#### osm_background
| ID* | OSM_ID | TAGS |
|---|---|---|
| INTEGER | INTEGER | TEXT |

**ID**: PRIMARY KEY AUTOINCREMENT  
The unique identifier for the background OSM data record.

**OSM_ID**: NOT NULL UNIQUE  
The unique identifier of the OSM element.

**TAGS**: NOT NULL  
Tags associated with the OSM element in JSON format.

---


In [ ]:
# TYPE: 0: background, 1: area, 2: nonearea
OSM_DATA_TABLE_INIT_CMD = """CREATE TABLE IF NOT EXISTS osm_data (
                            ID          INTEGER     PRIMARY KEY AUTOINCREMENT,
                            PATCH_NAME  TEXT        NOT NULL,
                            OSM_ID      INTEGER     NOT NULL,
                            TAGS        TEXT,   
                            TYPE        TINYINT     NOT NULL,
                            GEOMETRY    TEXT,
                            DATA_DATE   DATE               
                            )"""
# create osm_background table
OSM_BACKGROUND_TABLE_INIT_CMD = """CREATE TABLE IF NOT EXISTS osm_background (
                                    ID          INTEGER     PRIMARY KEY AUTOINCREMENT,
                                    OSM_ID      INTEGER     NOT NULL UNIQUE,
                                    TAGS        TEXT        NOT NULL
                                    )"""
# init the data base
database_file = os.path.join(DB_ROOT,'osm.db')
conn = sqlite3.connect(database_file)

c_annotation = conn.cursor()
c_annotation.execute(OSM_DATA_TABLE_INIT_CMD)
c_annotation.execute(OSM_BACKGROUND_TABLE_INIT_CMD)

# insert data into osm_data table
# insert data into osm_background table
conn.commit()
conn.close()

# Create annotation_meta database for Task 3

## Create database for storing rewrite examples
This database is used to store raw rewrite data and their corresponding examples. The schema includes two tables: `rewrite_raw` and `rewrite_examples`.

### Table Definitions

#### rewrite_raw
| id* | patch_id | prompt_id | text |
|---|---|---|---|
| INTEGER | INTEGER | INTEGER | TEXT |

**id**: PRIMARY KEY AUTOINCREMENT  
The unique identifier for the raw rewrite record.

**patch_id**: NOT NULL  
The ID of the patch. This references the `id` column in the `patch` table of the metadata database.

**prompt_id**: NOT NULL  
The ID of the prompt. This references the `id` column in the `prompt` table of the annotation database.

**text**: NOT NULL  
The raw text associated with this rewrite record.

---

#### rewrite_examples
| id* | rewrite_raw_id | rewrite_text |
|---|---|---|
| INTEGER | INTEGER | TEXT |

**id**: PRIMARY KEY AUTOINCREMENT  
The unique identifier for the rewrite example record.

**rewrite_raw_id**: NOT NULL  
The ID of the associated raw rewrite record. This references the `id` column in the `rewrite_raw` table.

**rewrite_text**: NOT NULL  
The rewritten text example associated with the raw rewrite record.

---

### Notes
- The `patch_id` in the `rewrite_raw` table is a foreign key referencing the `patch` table in the metadata database.
- The `prompt_id` in the `rewrite_raw` table is a foreign key referencing the `prompt` table in the annotation database.
- This schema enables the storage and management of rewrite examples and their associations with specific patches and prompts.


In [ ]:
# create tables for storing rewrite examples
SQL_CREATE_REWRITE_RAW_TABLE = """ CREATE TABLE IF NOT EXISTS rewrite_raw (
                                        id integer PRIMARY KEY AUTOINCREMENT,
                                        patch_id integer NOT NULL,
                                        prompt_id integer NOT NULL,
                                        text text NOT NULL)"""
                                        
SQL_CREATE_REWRITE_EXAMPLES_TABLE = """ CREATE TABLE IF NOT EXISTS rewrite_examples (
                                        id integer PRIMARY KEY AUTOINCREMENT,
                                        rewrite_raw_id integer NOT NULL,
                                        rewrite_text text NOT NULL,
                                        FOREIGN KEY (rewrite_raw_id) REFERENCES rewrite_raw(id))"""
                                        
database_file = os.path.join(DB_ROOT,'annotation_meta.db')
conn = sqlite3.connect(database_file)
c = conn.cursor()

c.execute(SQL_CREATE_REWRITE_RAW_TABLE)
c.execute(SQL_CREATE_REWRITE_EXAMPLES_TABLE)

conn.commit()
conn.close()